# Packages


In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pandas import DataFrame
from pprint import pprint
import datetime

# Gensim
import gensim
import spacy
import logging
import warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import matplotlib.pyplot as plt
import mysql.connector
import pandas.io.sql as psql

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# Connection to SQL

In [2]:
conn = mysql.connector.connect(host="ba-isdsclass-dev2.lsu.edu",
                                user="pgutie2",
                                passwd="msadatabase",
                                database="msa_schema")
cursor = conn.cursor()

In [3]:
sql = ("Select * from text order by article_date asc;")
db = pd.read_sql(sql,conn)
db

,article_url,article_date,article_title,article_text,author,twitter,timestamp
0,http://blogs.nature.com/freeassociation/2015/0...,01 Apr 2015,Sidestepping spurious associations,SitemapNature Genetics | Free Association01 Ap...,Brooke LaFlamme,,2020-03-04 13:36:42
1,http://blogs.nature.com/methagora/2015/04/meth...,01 Apr 2015,Methods comparisons are hard,SitemapNature Methods | Methagora01 Apr 2015 |...,Natalie de Souza,,2020-03-04 13:36:43
2,http://blogs.nature.com/naturejobs/2016/04/01/...,01 Apr 2016,Most read on Naturejobs: March 2016,SitemapNaturejobs | Naturejobs Blog01 Apr 2016...,Jack Leeming,,2020-03-04 13:36:11
3,http://blogs.nature.com/naturejobs/2016/08/01/...,01 Aug 2016,Five tips to overcome authorship conflict,SitemapNaturejobs | Naturejobs Blog01 Aug 2016...,Jack Leeming,,2020-03-04 13:36:01
4,http://blogs.nature.com/indigenus/2017/08/natu...,01 Aug 2017,Nature India Photo Story: Cubanacan the Litigon,SitemapNature India | Indigenus01 Aug 2017 | 0...,Subhra Priyadarshini,,2020-03-04 13:35:25
5,http://blogs.nature.com/naturejobs/2017/08/01/...,01 Aug 2017,TechBlog: Jupyter Joins the Galaxy,SitemapNaturejobs | Naturejobs Blog01 Aug 2017...,Jeffrey Perkel,,2020-03-04 13:35:25
6,http://blogs.nature.com/ofschemesandmemes/2018...,01 Aug 2018,Nature Research journals trial new tools to en...,SitemapOf Schemes and Memes Blog01 Aug 2018 | ...,Mark Staniland,,2020-03-04 13:35:02
7,http://blogs.nature.com/naturejobs/2018/08/01/...,01 Aug 2018,Announcement: The Naturejobs blog is moving house,SitemapNaturejobs | Naturejobs Blog01 Aug 2018...,Jack Leeming,,2020-03-04 13:35:02
8,http://blogs.nature.com/naturejobs/2014/12/01/...,01 Dec 2014,How important is it to move between academic i...,SitemapNaturejobs | Naturejobs Blog01 Dec 2014...,Julie Gould,,2020-03-04 13:36:55
9,http://blogs.nature.com/blog/uncovering-the-se...,01 Dec 2014,Uncovering the secrets of the orchid,Sitemapnature.com blogs01 Dec 2014 | 16:28 GMT...,Brooke LaFlamme,,2020-03-04 13:36:55


# Data Processing

In [4]:
def sent_to_words(sentences):
    for sent in sentences:
        #sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        #sent = re.sub('\s+', ' ', sent)  # remove newline chars
        #sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  

# Convert to list
data = db.article_text.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['sitemapnature', 'genetics', 'free', 'association', 'apr', 'bst', 'posted', 'by', 'brooke', 'laflamme', 'category', 'genetic', 'association', 'layers', 'of', 'structure', 'robert', 'delaunay', 'premier', 'disquegenome', 'wide', 'association', 'tests', 'have', 'been', 'hugely', 'successful', 'at', 'finding', 'genes', 'and', 'even', 'specific', 'mutations', 'that', 'contribute', 'to', 'traits', 'ranging', 'from', 'human', 'height', 'to', 'schizophrenia', 'at', 'its', 'most', 'basic', 'the', 'idea', 'is', 'that', 'group', 'of', 'individuals', 'with', 'shared', 'phenotype', 'should', 'also', 'share', 'some', 'genetic', 'variants', 'in', 'common', 'that', 'are', 'causally', 'related', 'to', 'the', 'trait', 'in', 'question', 'unfortunately', 'there', 'are', 'other', 'reasons', 'that', 'individuals', 'who', 'share', 'trait', 'such', 'as', 'cardiovascular', 'disease', 'or', 'epilepsy', 'might', 'share', 'genetic', 'variants', 'in', 'common', 'for', 'example', 'gene', 'might', 'seem', 'to', '

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)



from spacy.lang.en import English

#!python -m spacy download en  # run in terminal once
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load(r'C:\Users\parke\Anaconda3\Lib\site-packages\en_core_web_sm\en_core_web_sm-2.2.5', disable=['parser', 'ner'])
    # nlp = spacy.load(r'C:\Users\imget\Anaconda3\Lib\site-packages\en_core_web_lg\en_core_web_lg-2.2.5', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for  word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)  # processed Text Data!

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)

In [ ]:
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]

# Coherence Calculation

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_ready, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
ldamodel = LdaModel(corpus=corpus, num_topics=8, id2word=id2word)
ldamodel.show_topics(8)

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary=ldamodel.id2word)
vis

In [ ]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel, corpus=corpus, texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

In [ ]:
c = conn.cursor()

In [ ]:
c.execute('Create Table topics (Document_No int, Topic_Num int, Topic_Perc_Contrib double, Keywords text, Text text)')
conn.commit()

In [ ]:
df_dominant_topic.to_sql('topics', conn, if_exists = 'replace', index = False)

In [ ]:

# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)